# Elo ratingを実験してみる
自分の頭で実装する！

参考：https://qiita.com/kenta_eros/items/960ba93fceaa5fddfda5

## モジュールのインポート
その他雑多なものも。

In [1]:
import random

## データを用意
雑だけど、配列を用意しておく。

In [15]:
# 人とレートの組み合わせ（rateが低い方が弱い）
rating = [
          {"key": "A", "rate": 200},
          {"key": "B", "rate": 100},
          {"key": "C", "rate": 50},
          {"key": "D", "rate": 250},
          {"key": "E", "rate": 300},
          {"key": "F", "rate": 100},
          {"key": "G", "rate": 75},
]

In [3]:
def lograte(player1, player2, rating):
  print(player1["key"] + "のレート: " + str(player1["rate"]))
  print(player2["key"] + "のレート: " + str(player2["rate"]))

In [4]:
def logratelist(rating):
  print("レート一覧")
  for player in rating:
    print(player["key"] + ": " + str(player["rate"]))

## 勝率の計算
対戦する2人のプレイヤーのレートにより、対戦の勝率は変わる。具体的には、プレイヤー$A, B$のレートをそれぞれ$R_A, R_B$とすると、$A$が$B$に勝つ勝率$W_{AB}$は

$$
  W_{AB} = \frac{1}{10^{(R_A-R_B)/400} + 1}
$$
となる。

In [5]:
def calc_winrate(Arate, Brate):
  sa = Arate - Brate
  return 1 / (10 ** (sa / 400) + 1)

## 対戦後のレートの変化
対戦後のレートの変化は、計算した勝率によって変化する。直感的には、

- 格上に勝った場合レートが大きく上がるが、格下に勝ってもレートの上り幅は小さい。
- 格上に負けてもレートは大きくは下がらないが、格下に負けると大きく下がってしまう。

ような仕組みが良い。Elo ratingでは、対戦後の$A, B$の勝率を$R_A', R_B'$とすると、$AがB$に勝利した時

$$
  R_A' = R_A + K (1 - W_{AB}) \\
  R_B' = R_B - K W_{BA}
$$

となる。

In [6]:
def moverate(winner, looser, prob, K, rating):
  winner["rate"] = winner["rate"] + K * (1 - prob)
  looser["rate"] = looser["rate"] - K * (1 - prob)

## 対戦させてみる
試しに対戦させてみて、レートの変動具合を確認してみる。

In [12]:
def battle(p1, p2, rating):
  K = 32
  player1 = rating[p1]
  player2 = rating[p2]
  print(player1["key"] + "と" + player2["key"] + "が対戦します")
  lograte(player1, player2, rating)
  winrate12 = calc_winrate(player1["rate"], player2["rate"])
  winrate21 = calc_winrate(player2["rate"], player1["rate"])
  seed = random.random()
  if seed <= winrate12:
    print(player1["key"] + "の勝利")
    moverate(player1, player2, winrate12, K, rating)
    lograte(player1, player2, rating)
  else:
    print(player2["key"] + "の勝利")
    moverate(player2, player1, winrate21, K, rating)
    lograte(player1, player2, rating)

In [13]:
def main(rating):
  logratelist(rating)
  howmany = int(input("何回やる？ => "))
  for i in range(howmany):
    p1num = random.randint(0, len(rating) - 1)
    p2num = random.randint(0, len(rating) - 1)
    battle(p1num, p2num, rating)
  print("\n----------\nゲーム終了")
  logratelist(rating)

In [16]:
main(rating)

レート一覧
A: 200
B: 100
C: 50
D: 250
E: 300
F: 100
G: 75
何回やる？ => 20
DとDが対戦します
Dのレート: 250
Dのレート: 250
Dの勝利
Dのレート: 250.0
Dのレート: 250.0
AとCが対戦します
Aのレート: 200
Cのレート: 50
Cの勝利
Aのレート: 190.50832011109853
Cのレート: 59.49167988890149
FとBが対戦します
Fのレート: 100
Bのレート: 100
Bの勝利
Fのレート: 84.0
Bのレート: 116.0
DとBが対戦します
Dのレート: 250.0
Bのレート: 116.0
Dの勝利
Dのレート: 271.88212205419705
Bのレート: 94.11787794580292
CとCが対戦します
Cのレート: 59.49167988890149
Cのレート: 59.49167988890149
Cの勝利
Cのレート: 59.49167988890149
Cのレート: 59.49167988890149
FとGが対戦します
Fのレート: 84.0
Gのレート: 75
Fの勝利
Fのレート: 100.41437263657164
Gのレート: 58.58562736342836
EとCが対戦します
Eのレート: 300
Cのレート: 59.49167988890149
Cの勝利
Eのレート: 293.5906909684226
Cのレート: 65.90098892047887
BとEが対戦します
Bのレート: 94.11787794580292
Eのレート: 293.5906909684226
Bの勝利
Bのレート: 101.82371616142515
Eのレート: 285.8848527528004
BとAが対戦します
Bのレート: 101.82371616142515
Aのレート: 190.50832011109853
Bの勝利
Bのレート: 113.8260863014115
Aのレート: 178.5059499711122
EとEが対戦します
Eのレート: 285.8848527528004
Eのレート: 285.8848527528004
Eの勝利
Eのレート: 285.8848527528004
Eのレー

Elo ratingを使えばいろいろなレーティングができそうですね。